# 股票成交量变化预测

In [1]:
import pandas as pd

import torch
import torch.nn
import torch.optim

数据读取

In [2]:
url = 'http://raw.githubusercontent.com/zhiqingxiao/pytorch-book/master/chapter06_logistic/FB.csv'
df = pd.read_csv(url, index_col=0)
df

,Open,High,Low,Close,Volume,ExDividend,SplitRatio,AdjOpen,AdjHigh,AdjLow,AdjClose,AdjVolume
Date,,,,,,,,,,,,
2018-03-27,156.310,162.8500,150.7500,152.1900,76787884.0,0.0,1.0,156.310,162.8500,150.7500,152.1900,76787884.0
2018-03-26,160.820,161.1000,149.0200,160.0600,125438294.0,0.0,1.0,160.820,161.1000,149.0200,160.0600,125438294.0
2018-03-23,165.440,167.1000,159.0200,159.3900,52306891.0,0.0,1.0,165.440,167.1000,159.0200,159.3900,52306891.0
2018-03-22,166.130,170.2700,163.7200,164.8900,73389988.0,0.0,1.0,166.130,170.2700,163.7200,164.8900,73389988.0
2018-03-21,164.800,173.4000,163.3000,169.3900,105350867.0,0.0,1.0,164.800,173.4000,163.3000,169.3900,105350867.0
2018-03-20,167.470,170.2000,161.9500,168.1500,128925534.0,0.0,1.0,167.470,170.2000,161.9500,168.1500,128925534.0
2018-03-19,177.010,177.1700,170.0600,172.5600,86897749.0,0.0,1.0,177.010,177.1700,170.0600,172.5600,86897749.0
2018-03-16,184.490,185.3300,183.4100,185.0900,23090480.0,0.0,1.0,184.490,185.3300,183.4100,185.0900,23090480.0
2018-03-15,183.240,184.0000,182.1900,183.8600,15461869.0,0.0,1.0,183.240,184.0000,182.1900,183.8600,15461869.0


In [3]:
train_start, train_end = sum(df.index >= '2017'), sum(df.index >= '2013')
test_start, test_end = sum(df.index >= '2018'), sum(df.index >= '2017')
n_total_train = train_end - train_start
n_total_test = test_end - test_start
s_mean = df[train_start:train_end].mean()
s_std = df[train_start:train_end].std()
n_features = 5
df_feature = ((df - s_mean) / s_std).iloc[:, :n_features]
s_label = (df['Volume'] < df['Volume'].shift(1)).astype(int)
df_feature, s_label

(                Open      High       Low     Close     Volume
 Date                                                         
 2018-03-27  2.526504  2.693290  2.395682  2.395569   1.267472
 2018-03-26  2.671127  2.637488  2.339793  2.648029   2.944486
 2018-03-23  2.819278  2.828808  2.662850  2.626536   0.423596
 2018-03-22  2.841404  2.929889  2.814687  2.802970   1.150344
 2018-03-21  2.798755  3.029695  2.801118  2.947325   2.252058
 2018-03-20  2.884374  2.927657  2.757506  2.907547   3.064693
 2018-03-19  3.190296  3.149908  3.019505  3.049015   1.615966
 2018-03-16  3.430159  3.410103  3.450785  3.450963  -0.583514
 2018-03-15  3.390075  3.367694  3.411372  3.411506  -0.846478
 2018-03-14  3.369552  3.375666  3.400388  3.422092  -0.813214
 2018-03-13  3.466075  3.431149  3.376482  3.347990  -0.767869
 2018-03-12  3.453889  3.434656  3.476953  3.440377  -0.866758
 2018-03-09  3.411560  3.415843  3.444324  3.455454  -0.760635
 2018-03-08  3.400337  3.380449  3.387466  3.362746  -0

训练和测试

In [4]:
fc = torch.nn.Linear(n_features, 1)
weights, bias = fc.parameters()
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(fc.parameters())

x = torch.tensor(df_feature.values, dtype=torch.float32)
y = torch.tensor(s_label.values.reshape(-1, 1), dtype=torch.float32)

n_step = 20001
for step in range(n_step):
    if step:
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    pred = fc(x)
    loss = criterion(pred[train_start:train_end], y[train_start:train_end])
    
    if step % 500 == 0:
        print('#{}, 损失 = {:g}'.format(step, loss))
        
        output = (pred > 0)
        correct = (output == y.byte())
        n_correct_train = correct[train_start:train_end].sum().item()
        n_correct_test = correct[test_start:test_end].sum().item()
        accuracy_train = n_correct_train / n_total_train
        accuracy_test = n_correct_test / n_total_test
        print('训练集准确率 = {}, 测试集准确率 = {}'.format(accuracy_train, accuracy_test))

#0, 损失 = 0.797557, 
训练集准确率 = 0.5059523809523809, 测试集准确率 = 0.456
#500, 损失 = 0.660889, 
训练集准确率 = 0.623015873015873, 测试集准确率 = 0.564
#1000, 损失 = 0.658926, 
训练集准确率 = 0.6091269841269841, 测试集准确率 = 0.548
#1500, 损失 = 0.658904, 
训练集准确率 = 0.6101190476190477, 测试集准确率 = 0.548
#2000, 损失 = 0.658884, 
训练集准确率 = 0.6101190476190477, 测试集准确率 = 0.548
#2500, 损失 = 0.658859, 
训练集准确率 = 0.6091269841269841, 测试集准确率 = 0.548
#3000, 损失 = 0.658825, 
训练集准确率 = 0.6091269841269841, 测试集准确率 = 0.548
#3500, 损失 = 0.658782, 
训练集准确率 = 0.6081349206349206, 测试集准确率 = 0.548
#4000, 损失 = 0.658726, 
训练集准确率 = 0.6081349206349206, 测试集准确率 = 0.548
#4500, 损失 = 0.658655, 
训练集准确率 = 0.6101190476190477, 测试集准确率 = 0.548
#5000, 损失 = 0.658565, 
训练集准确率 = 0.6101190476190477, 测试集准确率 = 0.548
#5500, 损失 = 0.658451, 
训练集准确率 = 0.6101190476190477, 测试集准确率 = 0.548
#6000, 损失 = 0.658308, 
训练集准确率 = 0.6101190476190477, 测试集准确率 = 0.548
#6500, 损失 = 0.658131, 
训练集准确率 = 0.6111111111111112, 测试集准确率 = 0.548
#7000, 损失 = 0.657913, 
训练集准确率 = 0.6121031746031746, 测试集准确率 = 0.548
